## Build a Basic ChatBot using LangGraph GraphAPI

In [1]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph,START,END

from langgraph.graph.message import add_messages

In [2]:
class State(TypedDict):

    # messages have the  type lst.The add_messages function 
    # in the annotation how this state key should be updated
    messages:Annotated[list,add_messages]

graph_builder=StateGraph(State)

In [3]:
graph_builder

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

#load data


True

In [5]:
# Define LLM
from langchain_groq import ChatGroq
from langchain.chat_models import init_chat_model


#initilize LLM
llm=ChatGroq(model="llama-3.1-8b-instant")

In [6]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001486A2527B0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001486A253380>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
llm=init_chat_model("groq:llama-3.1-8b-instant")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001486A3C8050>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001486A3C8A50>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
# Node Functionality

def chatbot(state:State):
    return {"messages":[llm.invoke(state["messages"])]}

In [9]:
graph_builder=StateGraph(State)

#Adding Node
graph_builder.add_node("llmchatbot",chatbot)

# Add Edges
graph_builder.add_edge(START,"llmchatbot")
graph_builder.add_edge("llmchatbot",END)

# compile the graph

graph = graph_builder.compile()

In [10]:
graph

ValueError: Failed to reach https://mermaid.ink/ API while trying to render your graph. Status code: 400.

To resolve this issue:
1. Check your internet connection and try again
2. Try with higher retry settings: `draw_mermaid_png(..., max_retries=5, retry_delay=2.0)`
3. Use the Pyppeteer rendering method which will render your graph locally in a browser: `draw_mermaid_png(..., draw_method=MermaidDrawMethod.PYPPETEER)`

In [11]:
response=graph.invoke({"messages":"Hi"})

In [12]:
response["messages"]

[HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}, id='8319a7a5-c8e1-4752-b2ba-d7a35b80dba1'),
 AIMessage(content='How can I assist you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 36, 'total_tokens': 44, 'completion_time': 0.010487327, 'prompt_time': 0.002585579, 'queue_time': 0.223421507, 'total_time': 0.013072906}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_3ddc9808b3', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--fe7c135a-647b-4b5c-8610-5352458b9334-0', usage_metadata={'input_tokens': 36, 'output_tokens': 8, 'total_tokens': 44})]

In [13]:
response["messages"][-1].content

'How can I assist you today?'

In [14]:
for event in graph.stream({"messages":"Hi , How are you ?"}):
    for value in event.values():
        print(value["messages"][-1].content)

I'm just a language model, so I don't have emotions or feelings like humans do, but I'm functioning properly and ready to assist you. How can I help you today?


ChatBot with Tools

In [15]:
from langchain_tavily import TavilySearch

tool = TavilySearch(max_results=2)
# ans = tool.invoke("what is langGraph")
tool.invoke("what is langGraph")

{'query': 'what is langGraph',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://www.datacamp.com/tutorial/langgraph-tutorial',
   'title': 'LangGraph Tutorial: What Is LangGraph and How to Use It?',
   'content': 'LangGraph is a library within the LangChain ecosystem that provides a framework for defining, coordinating, and executing multiple LLM agents (or chains) in a structured and efficient manner. By managing the flow of data and the sequence of operations, LangGraph allows developers to focus on the high-level logic of their applications rather than the intricacies of agent coordination. Whether you need a chatbot that can handle various types of user requests or a multi-agent system that performs complex tasks, LangGraph provides the tools to build exactly what you need. LangGraph significantly simplifies the development of complex LLM applications by providing a structured framework for managing state and coordinating agent interaction

In [ ]:
# ans["results"][-1]["content"]

'* LangGraph is a library built on top of Langchain that is designed to facilitate the creation of cyclic graphs for large language model (LLM) – based AI agents. * Langgraph introduces a chat agent executor that represents the agent state as a list of messages, which is particularly useful for newer, chat-based models. The agent executor class in the Langchain framework was the main tool for building and executing AI agents before LangGraph. Large Language Models (LLMs) are the foundation for designing sophisticated AI agents, and LangGraph, built on top of Langchain, is intended to make the process of creating cyclic graphs easier. Ans. LangGraph addresses the limitations of previous AI agent development frameworks by providing more flexibility, better state management, and support for cyclic execution and multi-agent systems.'

In [32]:
## Define custom function

def multiply(a:int,b:int)->int:

    """Multiply a and b

    Args:
        a (int) : first int
        b (int) : second int

    Returns:
        int : output int   
    """

    return a*b

In [33]:
tools=[tool,multiply]

In [34]:
llm_with_tool=llm.bind_tools(tools)

In [35]:
llm_with_tool

RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001486A3C8050>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001486A3C8A50>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'tavily_search', 'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. It not only retrieves URLs and snippets, but offers advanced search depths, domain management, time range filters, and image search, this tool delivers real-time, accurate, and citation-backed results.Input should be a search query.', 'parameters': {'properties': {'query': {'description': 'Search query to look up', 'type': 'string'}, 'include_domains': {'anyOf': [{'items': {'type': 'string'}, 'type': 'array'}, {'type': 'null'}], 'default': [], 'description': 'A

In [36]:
## Build entire State Graph

from langgraph.graph import StateGraph,START,END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

## Node Defination

def tool_calling_llm(state:State):
    return {"messages":[llm_with_tool.invoke(state["messages"])]}

## Graph

builder= StateGraph(State)
builder.add_node("tool_calling_llm",tool_calling_llm)
builder.add_node("tools",ToolNode(tools))

## Add Edge

builder.add_edge(START,"tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    tools_condition
)
builder.add_edge("tools",END)


## Compile the graph

graph = builder.compile()


# from IPython.display import Image,display
# display(Image(graph.get_graph().draw_mermaid_png()))

In [37]:
response = graph.invoke({"messages":"What is recent H1B news"})

In [ ]:
response['messages'][-1].content

'{"query": "H1B news", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.devdiscourse.com/article/headlines/3634541-implications-of-h-1b-visa-changes-on-south-korean-workforce", "title": "Implications of H-1B Visa Changes on South Korean Workforce - Devdiscourse", "score": 0.7316155, "published_date": "Sat, 20 Sep 2025 09:18:43 GMT", "content": "+ NEWS * NEWS * News # Implications of H-1B Visa Changes on South Korean Workforce ## South Korea will evaluate the effects of the U.S. H-1B visa changes on its companies and workers. The South Korean foreign ministry has announced an assessment of the recent changes to the U.S. H-1B visa program and their potential impact on South Korean enterprises and professionals aspiring to work in the United States. On Friday, the Trump administration revealed a proposal requiring companies to pay a $100,000 annual fee for each H-1B worker visa. * H-1B Visa #### US H-1B Visa Fee Surge Threatens India\'s Tech Sect

: 

In [38]:
for m in response['messages']:
    m.pretty_print()

================================ Human Message =================================

What is recent H1B news
================================== Ai Message ==================================
Tool Calls:
  tavily_search (kpp3ryyd9)
 Call ID: kpp3ryyd9
  Args:
    query: H1B news
    time_range: month
    topic: news
================================= Tool Message =================================
Name: tavily_search

{"query": "H1B news", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.devdiscourse.com/article/headlines/3634541-implications-of-h-1b-visa-changes-on-south-korean-workforce", "title": "Implications of H-1B Visa Changes on South Korean Workforce - Devdiscourse", "score": 0.7316155, "published_date": "Sat, 20 Sep 2025 09:18:43 GMT", "content": "+ NEWS * NEWS * News # Implications of H-1B Visa Changes on South Korean Workforce ## South Korea will evaluate the effects of the U.S. H-1B visa changes on its companies and workers. The South Ko

In [39]:
response=graph.invoke({"messages":"what is 2 multiply by  3"})

In [40]:
for m in response['messages']:
    m.pretty_print()

================================ Human Message =================================

what is 2 multiply by  3
================================== Ai Message ==================================
Tool Calls:
  multiply (c7p7j7saw)
 Call ID: c7p7j7saw
  Args:
    a: 2
    b: 3
================================= Tool Message =================================
Name: multiply

6
